In [70]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import math

In [71]:
'''READS IN TEAM STATS FOR'''
team_stats1314 = pd.read_csv('./preprocessed/team_stats1314.csv', sep=',')
team_stats1415 = pd.read_csv('./preprocessed/team_stats1415.csv', sep=',')
team_stats1516 = pd.read_csv('./preprocessed/team_stats1516.csv', sep=',')
team_stats1617 = pd.read_csv('./preprocessed/team_stats1617.csv', sep=',')
team_stats1718 = pd.read_csv('./preprocessed/team_stats1718.csv', sep=',')

dfs = [team_stats1314, team_stats1415, team_stats1516, team_stats1617, team_stats1718]

for df in dfs:
    df['Team'] = df.Team.str.replace(' ', '_')
    df['Team'] = df['Team'].str[:-1]

team_stats1415


,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
0,0,1,Golden_State_Warriors,82,240.6,41.6,87.0,0.478,10.8,27.0,...,10.4,34.3,44.7,27.4,9.3,6.0,14.5,19.9,110.0,81.7
1,1,4,LA_Clippers,82,240.6,39.4,83.3,0.473,10.1,26.9,...,9.6,33.1,42.6,24.8,7.8,5.0,12.3,21.3,106.7,68.3
2,2,9,Dallas_Mavericks,82,242.4,39.7,85.8,0.463,8.9,25.4,...,10.5,31.8,42.3,22.5,8.1,4.5,13.0,20.0,105.2,61.0
3,3,11,Toronto_Raptors,82,242.1,37.9,83.3,0.455,8.9,25.1,...,10.7,30.8,41.5,20.7,7.5,4.4,12.9,20.9,104.0,59.8
4,4,3,Houston_Rockets,82,241.5,37.0,83.3,0.444,11.4,32.7,...,11.7,32.0,43.7,22.2,9.5,5.0,16.7,22.0,103.9,68.3
5,5,5,San_Antonio_Spurs,82,243.4,39.1,83.6,0.468,8.3,22.5,...,9.8,33.8,43.6,24.4,8.0,5.4,14.0,19.1,103.2,67.1
6,6,7,Cleveland_Cavaliers,82,241.2,37.7,82.2,0.458,10.1,27.5,...,11.1,31.9,43.0,22.1,7.4,4.1,14.3,18.4,103.1,64.6
7,7,8,Portland_Trail_Blazers,82,242.1,38.7,86.0,0.450,9.8,27.2,...,10.7,35.1,45.9,21.9,6.4,4.5,13.6,18.2,102.8,62.2
8,8,2,Atlanta_Hawks,82,240.6,38.1,81.7,0.466,10.0,26.2,...,8.7,31.8,40.6,25.7,9.1,4.6,14.2,17.8,102.5,73.2
9,9,16,Boston_Celtics,82,242.4,38.9,87.9,0.443,8.0,24.6,...,11.1,32.7,43.8,24.5,8.2,3.6,13.8,21.2,101.4,48.8


In [72]:
'''READS IN TEAM STATS AGAINST'''
opp_stats1314 = pd.read_csv('./preprocessed/opp_stats1314.csv', sep=',')
opp_stats1415 = pd.read_csv('./preprocessed/opp_stats1415.csv', sep=',')
opp_stats1516 = pd.read_csv('./preprocessed/opp_stats1516.csv', sep=',')
opp_stats1617 = pd.read_csv('./preprocessed/opp_stats1617.csv', sep=',')
opp_stats1718 = pd.read_csv('./preprocessed/opp_stats1718.csv', sep=',')

opp_arr = [opp_stats1314, opp_stats1415, opp_stats1516, opp_stats1617, opp_stats1718]

for df in opp_arr:
    df['Team'] = df['Team'].str[:-1]
    

In [73]:
series_in1314 = pd.read_csv('./Playoff_Matchups/Playoff_13-14.csv', sep=',')
series_in1415 = pd.read_csv('./Playoff_Matchups/Playoff_14-15.csv', sep=',')
series_in1516 = pd.read_csv('./Playoff_Matchups/Playoff_15-16.csv', sep=',')
series_in1617 = pd.read_csv('./Playoff_Matchups/Playoff_16-17.csv', sep=',')
series_in1314

,TeamA,TeamB,Winner
0,Indiana_Pacers,Atlanta_Hawks,TeamA
1,Chicago_Bulls,Washington_Wizards,TeamB
2,Toronto_Raptors,Brooklyn_Nets,TeamB
3,Miami_Heat,Charlotte_Bobcats,TeamA
4,San_Antonio_Spurs,Dallas_Mavericks,TeamA
5,Houston_Rockets,Portland_Trail_Blazers,TeamB
6,LA_Clippers,Golden_State_Warriors,TeamA
7,Oklahoma_City_Thunder,Memphis_Grizzlies,TeamA
8,Indiana_Pacers,Washington_Wizards,TeamA
9,Miami_Heat,Brooklyn_Nets,TeamA


In [74]:
def make_series(tm, series, df):
    
    if tm == 'a':
        a = series['TeamA'].tolist()
    else:
        a = series['TeamB'].tolist()
            
    team = pd.DataFrame()
    
    for x in a:
        to_concat = df[df['Team'] == x]
        frames = [team, to_concat]
        team = pd.concat(frames)
    
    return team
    

In [75]:
'''THIS MAKES THE FOR AND OPPOSING TABLES FOR THE PLAYOFFS TEAM A'''
a = 'a'
team_a1314 = make_series(a, series_in1314, team_stats1314)
team_a_opp1314 = make_series(a, series_in1314, opp_stats1314)
team_a1415 = make_series(a, series_in1415, team_stats1415)
team_a_opp1415 = make_series(a, series_in1415, opp_stats1415)
team_a1516 = make_series(a, series_in1516, team_stats1516)
team_a_opp1516 = make_series(a, series_in1516, opp_stats1516)
team_a1617 = make_series(a, series_in1617, team_stats1617)
team_a_opp1617 = make_series(a, series_in1617, opp_stats1617)
# team_stats1617
# team_a_opp1617


In [76]:
'''THIS MAKES THE FOR AND OPPOSING TABLES FOR THE PLAYOFFS TEAM B'''
b = 'b'
team_b1314 = make_series(b, series_in1314, team_stats1314)
team_b_opp1314 = make_series(b, series_in1314, opp_stats1314)
team_b1415 = make_series(b, series_in1415, team_stats1415)
team_b_opp1415 = make_series(b, series_in1415, opp_stats1415)
team_b1516 = make_series(b, series_in1516, team_stats1516)
team_b_opp1516 = make_series(b, series_in1516, opp_stats1516)
team_b1617 = make_series(b, series_in1617, team_stats1617)
team_b_opp1617 = make_series(b, series_in1617, opp_stats1617)

team_b1516

,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
13,13,12,Detroit_Pistons,82,242.4,37.9,86.4,0.439,9.0,26.2,...,12.5,33.9,46.3,19.4,7.0,3.7,13.5,19.0,102.0,53.7
3,3,7,Boston_Celtics,82,241.2,39.2,89.2,0.439,8.7,26.1,...,11.6,33.3,44.9,24.2,9.2,4.2,13.7,21.9,105.7,58.5
8,8,7,Charlotte_Hornets,82,242.1,37.0,84.4,0.439,10.6,29.4,...,9.0,35.0,43.9,21.7,7.3,5.3,12.6,18.1,103.4,58.5
12,12,11,Indiana_Pacers,82,242.4,38.3,85.2,0.450,8.1,23.0,...,10.3,33.9,44.2,21.2,9.0,4.8,14.9,20.0,102.2,54.9
2,2,17,Houston_Rockets,82,241.8,37.7,83.5,0.452,10.7,30.9,...,11.3,31.7,43.1,22.2,10.0,5.2,15.9,21.8,106.5,50.0
4,4,12,Portland_Trail_Blazers,82,241.5,38.6,85.9,0.450,10.5,28.5,...,11.6,33.9,45.5,21.3,6.9,4.6,14.6,21.7,105.1,53.7
11,11,14,Dallas_Mavericks,82,244.0,37.4,84.1,0.444,9.8,28.6,...,9.2,33.9,43.1,22.1,6.8,3.7,12.8,19.5,102.3,51.2
15,15,14,Memphis_Grizzlies,82,241.8,36.8,83.6,0.440,6.1,18.5,...,11.2,30.5,41.6,20.7,8.8,4.3,13.3,21.7,99.1,51.2
9,9,7,Atlanta_Hawks,82,241.8,38.6,84.4,0.458,9.9,28.4,...,8.3,33.8,42.1,25.6,9.1,5.9,15.0,19.1,102.8,58.5
10,10,4,Toronto_Raptors,82,241.2,36.7,81.3,0.451,8.6,23.4,...,10.2,33.2,43.4,18.7,7.8,5.5,13.1,19.6,102.7,68.3


In [ ]:
'''THIS MAKES THE WINNER DATAFRAME'''
winner1314 = series_in1314['Winner']
winner1415 = series_in1415['Winner']
winner1516 = series_in1516['Winner']
winner1617 = series_in1617['Winner']

In [ ]:
'''WRITES TEAM A STATS TO CSV FILES'''
team_a1314.to_csv('./preprocessed/final/playoffs_for_a_1314.csv')
team_a1415.to_csv('./preprocessed/final/playoffs_for_a_1415.csv')
team_a1516.to_csv('./preprocessed/final/playoffs_for_a_1516.csv')
team_a1617.to_csv('./preprocessed/final/playoffs_for_a_1617.csv')

team_a_opp1314.to_csv('./preprocessed/final/playoffs_opp_a_1314.csv')
team_a_opp1415.to_csv('./preprocessed/final/playoffs_opp_a_1415.csv')
team_a_opp1516.to_csv('./preprocessed/final/playoffs_opp_a_1516.csv')
team_a_opp1617.to_csv('./preprocessed/final/playoffs_opp_a_1617.csv')

In [ ]:
'''WRITES TEAM B STATS TO CSV FILES'''
team_b1314.to_csv('./preprocessed/final/playoffs_for_b_1314.csv')
team_b1415.to_csv('./preprocessed/final/playoffs_for_b_1415.csv')
team_b1516.to_csv('./preprocessed/final/playoffs_for_b_1516.csv')
team_b1617.to_csv('./preprocessed/final/playoffs_for_b_1617.csv')

team_b_opp1314.to_csv('./preprocessed/final/playoffs_opp_b_1314.csv')
team_b_opp1415.to_csv('./preprocessed/final/playoffs_opp_b_1415.csv')
team_b_opp1516.to_csv('./preprocessed/final/playoffs_opp_b_1516.csv')
team_b_opp1617.to_csv('./preprocessed/final/playoffs_opp_b_1617.csv')

In [ ]:
'''WRITES WINNER DATAFRAME TO CSV FILES'''
winner1314.to_csv('./preprocessed/final/winner1314.csv')
winner1415.to_csv('./preprocessed/final/winner1415.csv')
winner1516.to_csv('./preprocessed/final/winner1516.csv')
winner1617.to_csv('./preprocessed/final/winner1617.csv')